<a href="https://colab.research.google.com/github/robertoarturomc/ProgramacionConcurrente/blob/main/23_Introduccion_a_Spark_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Programación Concurrente
## 23. Introducción a Spark II



In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

En este Notebook, vamos a comparar el uso de algunas funciones en Pandas vs su equivalente en Pyspark. Para ello, crearemos dos DataFrames: uno de Pandas, y uno de Spark, que contengan la misma información.

In [ ]:
df_pandas = pd.read_csv('covidmex.csv')
df_pandas.head(5)


,ID_REGISTRO,ENTIDAD_RES,MUNICIPIO_RES,FECHA_INGRESO,FECHA_SINTOMAS,covidt,delta,lat,long,alt,qry,dayofyear,lengthofday
0,z526b3,9,12,2020-12-21,2020-12-18,1,3,19.201553,-99.201803,3008.946078,Cve_Ent==9 & Cve_Mun==12,356,11.768372
1,z3d1e2,9,5,2020-04-22,2020-04-20,1,2,19.482945,-99.113471,2229.000000,Cve_Ent==9 & Cve_Mun==5,113,13.441806
2,z21f6f,7,9,2020-04-27,2020-04-24,1,3,16.215993,-93.937585,83.173160,Cve_Ent==7 & Cve_Mun==9,118,13.399117
3,zz9040,19,45,2020-09-06,2020-09-03,1,3,26.069806,-100.277762,486.425316,Cve_Ent==19 & Cve_Mun==45,250,13.288479
4,z21c58,19,26,2020-07-10,2020-07-06,1,4,25.679632,-100.169543,551.750000,Cve_Ent==19 & Cve_Mun==26,192,14.507915


In [ ]:
df_spark = spark.read.csv('covidmex.csv', header=True)
df_spark.show(5)

+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|               lat|               long|               alt|                 qry|dayofyear|       lengthofday|
+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+
|     z526b3|          9|           12|   2020-12-21|    2020-12-18|     1|    3| 19.20155345588235| -99.20180252450989|3008.9460784313724|Cve_Ent==9 & Cve_...|      356|11.768371962176587|
|     z3d1e2|          9|            5|   2020-04-22|    2020-04-20|     1|    2|         19.482945|         -99.113471|            2229.0|Cve_Ent==9 & Cve_...|      113|13.441805501598786|
|     z21f6f|          7|            9|   2020-04-

¿Cómo imprimo los primeros n registros?

In [ ]:
df_spark.ID_REGISTRO.show(5)

TypeError: 'Column' object is not callable

In [ ]:
# Sólo funciona si es la última línea de mi celda. De lo contrario, tendría que usar la función display
df_pandas.head(5)

¿Cómo selecciono un subconjunto de columnas?

In [ ]:
# Para hacer operaciones
df_spark['ID_REGISTRO']

Column<'ID_REGISTRO'>

In [1]:
# Para hacer una subselección
df_spark.select('ID_REGISTRO').show(5)

NameError: name 'df_spark' is not defined

In [ ]:
df_spark.select(['ID_REGISTRO', 'delta']).show()

+-----------+-----+
|ID_REGISTRO|delta|
+-----------+-----+
|     z526b3|    3|
|     z3d1e2|    2|
|     z21f6f|    3|
|     zz9040|    3|
|     z21c58|    4|
|     z5c20f|    3|
|     z1d9a5|    5|
|     z33892|    2|
|     z22bc2|    1|
|     z1b053|    9|
|     z2a1ad|    4|
|     z4494e|    7|
|     zz8e57|    3|
|     z388cd|    5|
|     zz9079|    1|
|     z166d5|    3|
|     z2770b|    5|
|     z4f06b|    5|
|     z33a15|    2|
|     z49a69|    3|
+-----------+-----+
only showing top 20 rows



La función `col` ayuda a especificar que estamos seleccionando columnas.


In [ ]:
df_spark.select(col('ID_REGISTRO'))

DataFrame[ID_REGISTRO: string]

In [ ]:
df_spark.select(col('ID_REGISTRO'), col('delta'))

DataFrame[ID_REGISTRO: string, delta: string]

In [ ]:
# En Pandas hay distintas opciones
df_pandas['ID_REGISTRO']
df_pandas.ID_REGISTRO
df_pandas.loc[:,'ID_REGISTRO']
df_pandas[['ID_REGISTRO']]

¿Cómo creo una nueva columna?

Estos métodos funcionan tanto para crear nuevas columnas, como para sobreescribir columnas existentes.

In [ ]:
df_spark.withColumn('DoubleDelta', 2* df_spark['delta'])

DataFrame[ID_REGISTRO: string, ENTIDAD_RES: string, MUNICIPIO_RES: string, FECHA_INGRESO: string, FECHA_SINTOMAS: string, covidt: string, delta: string, lat: string, long: string, alt: string, qry: string, dayofyear: string, lengthofday: string, DoubleDelta: double]

In [ ]:
df_pandas['DoubleDelta'] = 2* df_pandas['delta']
df_pandas[['delta', 'DoubleDelta'] ]

,delta,DoubleDelta
0,3,6
1,2,4
2,3,6
3,3,6
4,4,8
...,...,...
5723136,0,0
5723137,6,12
5723138,5,10
5723139,3,6


¿Y si quiero crear una columna con un valor constante?

In [ ]:
# En Spark, hay que usar el operador lit. Esto, para agregar una capa extra de seguridad.
df_spark.withColumn('Pais', lit('Mexico')).show()

+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+------+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|               lat|               long|               alt|                 qry|dayofyear|       lengthofday|  Pais|
+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+------+
|     z526b3|          9|           12|   2020-12-21|    2020-12-18|     1|    3| 19.20155345588235| -99.20180252450989|3008.9460784313724|Cve_Ent==9 & Cve_...|      356|11.768371962176587|Mexico|
|     z3d1e2|          9|            5|   2020-04-22|    2020-04-20|     1|    2|         19.482945|         -99.113471|            2229.0|Cve_Ent==9 & Cve_...|      113|13.441805501598786|Mexico|
|     z21f6f|  

In [ ]:
# En Pandas es directo.
df_pandas['Pais'] = "Mexico"
df_pandas.head()

,ID_REGISTRO,ENTIDAD_RES,MUNICIPIO_RES,FECHA_INGRESO,FECHA_SINTOMAS,covidt,delta,lat,long,alt,qry,dayofyear,lengthofday,DoubleDelta,Pais
0,z526b3,9,12,2020-12-21,2020-12-18,1,3,19.201553,-99.201803,3008.946078,Cve_Ent==9 & Cve_Mun==12,356,11.768372,6,Mexico
1,z3d1e2,9,5,2020-04-22,2020-04-20,1,2,19.482945,-99.113471,2229.000000,Cve_Ent==9 & Cve_Mun==5,113,13.441806,4,Mexico
2,z21f6f,7,9,2020-04-27,2020-04-24,1,3,16.215993,-93.937585,83.173160,Cve_Ent==7 & Cve_Mun==9,118,13.399117,6,Mexico
3,zz9040,19,45,2020-09-06,2020-09-03,1,3,26.069806,-100.277762,486.425316,Cve_Ent==19 & Cve_Mun==45,250,13.288479,6,Mexico
4,z21c58,19,26,2020-07-10,2020-07-06,1,4,25.679632,-100.169543,551.750000,Cve_Ent==19 & Cve_Mun==26,192,14.507915,8,Mexico


¿Cómo renombro columnas?

In [ ]:
df_spark.withColumnRenamed('DoubleDelta', 'Doble_Delta')

DataFrame[ID_REGISTRO: string, ENTIDAD_RES: string, MUNICIPIO_RES: string, FECHA_INGRESO: string, FECHA_SINTOMAS: string, covidt: string, delta: string, lat: string, long: string, alt: string, qry: string, dayofyear: string, lengthofday: string]

In [ ]:
df_pandas.rename(columns={'DoubleDelta': 'Doble_Delta'}, inplace=True)

df_pandas

,ID_REGISTRO,ENTIDAD_RES,MUNICIPIO_RES,FECHA_INGRESO,FECHA_SINTOMAS,covidt,delta,lat,long,alt,qry,dayofyear,lengthofday,DoubleDelta,Pais
0,z526b3,9,12,2020-12-21,2020-12-18,1,3,19.201553,-99.201803,3008.946078,Cve_Ent==9 & Cve_Mun==12,356,11.768372,6,Mexico
1,z3d1e2,9,5,2020-04-22,2020-04-20,1,2,19.482945,-99.113471,2229.000000,Cve_Ent==9 & Cve_Mun==5,113,13.441806,4,Mexico
2,z21f6f,7,9,2020-04-27,2020-04-24,1,3,16.215993,-93.937585,83.173160,Cve_Ent==7 & Cve_Mun==9,118,13.399117,6,Mexico
3,zz9040,19,45,2020-09-06,2020-09-03,1,3,26.069806,-100.277762,486.425316,Cve_Ent==19 & Cve_Mun==45,250,13.288479,6,Mexico
4,z21c58,19,26,2020-07-10,2020-07-06,1,4,25.679632,-100.169543,551.750000,Cve_Ent==19 & Cve_Mun==26,192,14.507915,8,Mexico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5723136,m04599c,15,85,2021-10-04,2021-10-04,1,0,19.915901,-100.032044,2506.878788,Cve_Ent==15 & Cve_Mun==85,277,12.618177,0,Mexico
5723137,m12bac8,15,104,2021-10-26,2021-10-20,1,6,19.560023,-99.131272,2373.400000,Cve_Ent==15 & Cve_Mun==104,299,12.244050,12,Mexico
5723138,m17ebee,15,90,2022-01-13,2022-01-08,1,5,19.084046,-99.604780,2708.549020,Cve_Ent==15 & Cve_Mun==90,13,11.854991,10,Mexico
5723139,m190fc6,15,106,2022-02-28,2022-02-25,1,3,19.331082,-99.654851,2659.008621,Cve_Ent==15 & Cve_Mun==106,59,12.428171,6,Mexico


¿Y cómo elimino columnas ya creadas?


In [ ]:
df_spark.drop('Doble_Delta')

DataFrame[ID_REGISTRO: string, ENTIDAD_RES: string, MUNICIPIO_RES: string, FECHA_INGRESO: string, FECHA_SINTOMAS: string, covidt: string, delta: string, lat: string, long: string, alt: string, qry: string, dayofyear: string, lengthofday: string]

In [ ]:
df_pandas.drop(columns='Doble_Delta')

Para Agrupar, ¿cómo le puedo hacer?

In [2]:
# Nota cómo en Spark, se escribe con minúsculas y mayúsculas
df_spark.groupBy('ENTIDAD_RES').count().show(31)

NameError: name 'df_spark' is not defined

In [ ]:
# Mientras que, en Pandas, se escribe todo en minúscula.Además, si no hago algo adicional, ¡Me agrupa sobre todas las columnas!
df_pandas.groupby('ENTIDAD_RES').count()

,MUNICIPIO_RES
ENTIDAD_RES,
1,65685
2,133981
3,103629
4,34051
5,145465
6,54713
7,38900
8,131391
9,1395456


¿Cómo filtro algunos datos?

In [ ]:
df_spark.filter(col('ENTIDAD_RES')==9)

DataFrame[ID_REGISTRO: string, ENTIDAD_RES: string, MUNICIPIO_RES: string, FECHA_INGRESO: string, FECHA_SINTOMAS: string, covidt: string, delta: string, lat: string, long: string, alt: string, qry: string, dayofyear: string, lengthofday: string]

In [ ]:
df_pandas[df_pandas['ENTIDAD_RES']==9]

,ID_REGISTRO,ENTIDAD_RES,MUNICIPIO_RES,FECHA_INGRESO,FECHA_SINTOMAS,covidt,delta,lat,long,alt,qry,dayofyear,lengthofday,DoubleDelta,Pais
0,z526b3,9,12,2020-12-21,2020-12-18,1,3,19.201553,-99.201803,3008.946078,Cve_Ent==9 & Cve_Mun==12,356,11.768372,6,Mexico
1,z3d1e2,9,5,2020-04-22,2020-04-20,1,2,19.482945,-99.113471,2229.000000,Cve_Ent==9 & Cve_Mun==5,113,13.441806,4,Mexico
14,zz9079,9,7,2020-10-07,2020-10-06,1,1,19.359004,-99.092623,2242.000000,Cve_Ent==9 & Cve_Mun==7,281,12.550388,2,Mexico
16,z2770b,9,6,2020-09-08,2020-09-03,1,5,19.395901,-99.097613,2233.000000,Cve_Ent==9 & Cve_Mun==6,252,13.096421,10,Mexico
20,z482b8,9,12,2020-10-16,2020-10-16,1,0,19.201553,-99.201803,3008.946078,Cve_Ent==9 & Cve_Mun==12,290,12.398029,0,Mexico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5722877,m103826,9,5,2022-01-20,2022-01-17,1,3,19.482945,-99.113471,2229.000000,Cve_Ent==9 & Cve_Mun==5,20,11.891276,6,Mexico
5722886,m02450e,9,5,2022-01-27,2022-01-22,1,5,19.482945,-99.113471,2229.000000,Cve_Ent==9 & Cve_Mun==5,27,11.962798,10,Mexico
5722960,m004492,9,7,2022-02-01,2022-02-01,1,0,19.359004,-99.092623,2242.000000,Cve_Ent==9 & Cve_Mun==7,32,12.026781,0,Mexico
5722963,m06b5ea,9,10,2022-02-03,2022-02-02,1,1,19.310810,-99.271163,2862.500000,Cve_Ent==9 & Cve_Mun==10,34,12.053694,2,Mexico


¿Y cómo selecciono datos únicos de una columna?

In [ ]:
df_spark.select('covidt').distinct().show()

+------+
|covidt|
+------+
|     1|
+------+



In [ ]:
df_pandas['covidt'].unique()

array([1])

Por último, ¿cómo ordeno mis datos?

In [ ]:
df_spark.orderBy(col('FECHA_INGRESO')).show(10)

+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|               lat|               long|               alt|                 qry|dayofyear|       lengthofday|
+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+
|     044a42|          9|            5|   2020-02-27|    2020-02-23|     1|    4|         19.482945|         -99.113471|            2229.0|Cve_Ent==9 & Cve_...|       58|12.408267170291763|
|     05f3d3|          9|            7|   2020-02-27|    2020-02-22|     1|    5|         19.359004|         -99.092623|            2242.0|Cve_Ent==9 & Cve_...|       58|12.410717907324571|
|     135d2f|         13|           48|   2020-02-

In [ ]:
df_pandas.sort_values(by='FECHA_INGRESO').head(10)

,ID_REGISTRO,ENTIDAD_RES,MUNICIPIO_RES,FECHA_INGRESO,FECHA_SINTOMAS,covidt,delta,lat,long,alt,qry,dayofyear,lengthofday,DoubleDelta,Pais
14914,09b2c0,15,81,2020-02-27,2020-02-27,1,0,19.720484,-98.969380,2280.290909,Cve_Ent==15 & Cve_Mun==81,58,12.403571,0,Mexico
1436,044a42,9,5,2020-02-27,2020-02-23,1,4,19.482945,-99.113471,2229.000000,Cve_Ent==9 & Cve_Mun==5,58,12.408267,8,Mexico
1455,05f3d3,9,7,2020-02-27,2020-02-22,1,5,19.359004,-99.092623,2242.000000,Cve_Ent==9 & Cve_Mun==7,58,12.410718,10,Mexico
1471,135d2f,13,48,2020-02-27,2020-02-22,1,5,20.093813,-98.792376,2483.771429,Cve_Ent==13 & Cve_Mun==48,58,12.396192,10,Mexico
582,189846,15,104,2020-02-28,2020-02-19,1,9,19.560023,-99.131272,2373.400000,Cve_Ent==15 & Cve_Mun==104,59,12.423870,18,Mexico
1900,06e286,5,35,2020-02-28,2020-02-27,1,1,25.482478,-103.353042,1151.982456,Cve_Ent==5 & Cve_Mun==35,59,12.312812,2,Mexico
2042634,9b6ba5,15,13,2020-02-29,2020-02-29,1,0,19.559390,-99.284914,2420.285714,Cve_Ent==15 & Cve_Mun==13,60,12.441136,0,Mexico
2077,0aad01,7,101,2020-02-29,2020-02-25,1,4,16.708645,-93.136493,794.916667,Cve_Ent==7 & Cve_Mun==101,60,12.492054,8,Mexico
67,1d457b,11,20,2020-03-01,2020-03-01,1,0,21.089920,-101.645109,1911.365747,Cve_Ent==11 & Cve_Mun==20,61,12.432884,0,Mexico
3505652,d77d0d,20,26,2020-03-01,2020-02-28,1,2,16.993540,-97.550245,2394.035714,Cve_Ent==20 & Cve_Mun==26,61,12.501790,4,Mexico


Ahorita no podremos verlo, pero también puedo unir DataFrames

In [ ]:
#df_spark.union(df2)

In [ ]:
#pd.concat(df_pandas, df2)


## Diferencias entre Pandas y Spark.

- En Spark, TODO se hace con métodos. En Pandas, varias veces podemos (y tenemos que) trabajar directamente sobre los corchetes.
- En Spark, los nombres de Métodos y Funciones están directamente basados en los de SQL.
- En Pandas es recomendado (y a veces, necesario) andar guardando los Dataframes intermedios, ya sea con un nuevo nombre, o en el mismo DF, con el argumento `inplace=True`.
- Spark tiene _Lazy Evaluation_, por lo que en vez de guardar tablas intermedias, es mejor práctica encadenar varias instrucciones, usando los métodos correspondientes, y dejar que Spark Paralalelice y Optimice por sí solo.

Por cierto; Spark también permite código SQL, nmientras creemos Vistas Temporales. Eso sí, no es el método preferido: tanto por legibilidad, como por su tardanza extra con respecto al uso de funciones y métodos nativos de Spark.

Vamos a ver una serie de instruccciones, con el mismo resultado, pero generadas de distinta manera. Primero, con _Spark SQL_:

In [ ]:
result = spark.sql('''Select ID_REGISTRO, FECHA_INGRESO, 24 * lengthofday AS DURACION_HORAS FROM covid WHERE ENTIDAD_RES == 21 ORDER BY FECHA_INGRESO ASC''')

In [ ]:
df_spark.createOrReplaceTempView('covid')

# Nota el uso del Backslash (\) para dar saltos de línea.
result = spark.sql('''Select ID_REGISTRO, FECHA_INGRESO, 24 * lengthofday AS DURACION_HORAS \
                    FROM covid \
                    WHERE ENTIDAD_RES == 21\
                    ORDER BY FECHA_INGRESO ASC''')

result.show(5)

+-----------+-------------+------------------+
|ID_REGISTRO|FECHA_INGRESO|    DURACION_HORAS|
+-----------+-------------+------------------+
|     13b46c|   2020-03-11|303.40683237053224|
|     171804|   2020-03-12|303.83571112424437|
|     0409a5|   2020-03-12|  303.900256124863|
|     1b5fbf|   2020-03-13|304.27007431628704|
|     0e3b5d|   2020-03-13|304.27007431628704|
+-----------+-------------+------------------+
only showing top 5 rows



Luego, con Pandas. Nota cómo tengo que guardar también todas las tablas intermedias, con los riesgos que esto conlleva.
Tip: repasa el concepto de _Copias Profundas._

In [ ]:
df1 = df_pandas[['ID_REGISTRO', 'FECHA_INGRESO', 'lengthofday']]

df1['DURACION_HORAS'] = 24 * df1['lengthofday']

df2 = df1.drop(columns='lengthofday')

df3 = df2[df_pandas['ENTIDAD_RES'] == 21]

df4 = df3.sort_values(by='FECHA_INGRESO')

df4.head()

<ipython-input-158-6ebac9a2ae5b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['DURACION_HORAS'] = 24 * df1['lengthofday']


,ID_REGISTRO,FECHA_INGRESO,DURACION_HORAS
1650,13b46c,2020-03-11,303.406832
7500,0409a5,2020-03-12,303.900256
1755,171804,2020-03-12,303.835711
2123,1bdebf,2020-03-13,304.270074
2128,14a371,2020-03-13,304.270074


Otra vez en Pandas, pero usando `inplace` para evitar generar tantas tablas. Eso sí, esta configuración trae riesgos si trato de recuperar información de una tabla anterior.

In [ ]:
df_inplace = df_pandas[['ID_REGISTRO', 'FECHA_INGRESO', 'lengthofday']]

df_inplace['DURACION_HORAS'] = 24 * df_inplace['lengthofday']

df_inplace.drop(columns='lengthofday', inplace=True)

df_inplace = df_inplace[df_pandas['ENTIDAD_RES'] == 21]

df_inplace.sort_values(by='FECHA_INGRESO', inplace=True)

df_inplace.head()

<ipython-input-104-3e35b15e2302>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inplace['DURACION_HORAS'] = 24 * df_inplace['lengthofday']
<ipython-input-104-3e35b15e2302>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inplace.drop(columns='lengthofday', inplace=True)


,ID_REGISTRO,FECHA_INGRESO,DURACION_HORAS
1650,13b46c,2020-03-11,303.406832
7500,0409a5,2020-03-12,303.900256
1755,171804,2020-03-12,303.835711
2123,1bdebf,2020-03-13,304.270074
2128,14a371,2020-03-13,304.270074


Finalmente, la versión preferida: Usando Métodos de PySpark. Nota cómo encadenamos varios métodos, sin andar guardando DataFrames intermedios.

La _Lazy Evaluation_ de Spark hará que vaya acumulando todas estas instrucciones, y no las ejecutará mientras no haya un `.show()` o alguna otra función que me **obligue** a arrojar un resultado. En ese momento, Spark tomará todas mis instrucciones pendientes, paralelizará y optimizará el trabajo, calculará y me arrojará el resultado.

In [ ]:
df2 = df_spark.select(['ID_REGISTRO', 'FECHA_INGRESO', 'lengthofday'])\
          .withColumn('DURACION_HORAS', 24 * df_spark['lengthofday'])\
          .drop('lengthofday')\
          .filter(df_spark['ENTIDAD_RES'] == 21)\
          .orderBy(col('FECHA_INGRESO'))

In [ ]:
df2.show()

+-----------+-------------+------------------+
|ID_REGISTRO|FECHA_INGRESO|    DURACION_HORAS|
+-----------+-------------+------------------+
|     13b46c|   2020-03-11|303.40683237053224|
|     171804|   2020-03-12|303.83571112424437|
|     0409a5|   2020-03-12|  303.900256124863|
|     1b5fbf|   2020-03-13|304.27007431628704|
|     0e3b5d|   2020-03-13|304.27007431628704|
|     1bdebf|   2020-03-13|304.27007431628704|
|     14a371|   2020-03-13|304.27007431628704|
|     1a4ef4|   2020-03-14| 304.6959267107193|
|     137633|   2020-03-14| 304.7034469006758|
|     052993|   2020-03-14| 304.7045531782602|
|     1ae22e|   2020-03-15| 305.1400558047457|
|     04da81|   2020-03-15| 305.1400558047457|
|     1d3f3c|   2020-03-15|305.13713550351343|
|     199388|   2020-03-16| 305.5788326923569|
|     1af2a6|   2020-03-16| 305.5995077388841|
|     036788|   2020-03-16| 305.5788326923569|
|     1c2961|   2020-03-16| 305.5788326923569|
|     01acd0|   2020-03-17|306.01835959706835|
|     0e5965|

De **Tarea** Investigar:
- ¿Cómo cambio de tipo de Dato en Pyspark? (String a Int, por ejemplo)
- ¿Por qué querría cambiar de tipo de Dato, de manera manual?